# Hypothesis

This notebook serves as a demo of the basic features and functionality of the Python tool **Hypothesis**.

## What is Hypothesis

Hypothesis is a tool that allows us to write and generate tests that generate and allow for fuzzing over a large variety of data without actually manually making all the tests. The key features mentioned are **property-based-testing** using `@given`, **strategies** for primitive and more complex data types, **shrinking** to find the simplest possible counter-example, and **stateful testing** to test with defined rules and checking invariants.

## 1. Basic Usage: `@given` and Strategies

The core of Hypothesis is the `@given` decorator, which takes **strategies** that define how to generate data. Let's test a simple `add` function.

In [ ]:
from hypothesis import given, strategies as st

def add(a, b):
    """A simple function to add two numbers."""
    return a + b

# This is a property-based test!
# It states that for any two integers a and b, add(a, b) should equal a + b.

@given(a=st.integers(), b=st.integers())
def test_addition_property(a, b):
    print(f"Testing with a={a}, b={b}") 
    assert add(a, b) == a + b

test_addition_property()

Notice how Hypothesis tries a wide range of integers: positives, negatives, and zero. If the assertion had failed, Hypothesis would have reported it.

## 2. Shrinking, Counter-Examples, and Replaying

Hypothesis's superpower is finding the *simplest possible* failing example. This is called **shrinking**. Let's introduce a bug into a sorting function.

In [ ]:
import shutil
try:
    shutil.rmtree('.hypothesis')
except:
    print('no cache')


In [ ]:
from hypothesis import find, settings, note
from typing import List

def buggy_sort(numbers: List[int]) -> List[int]:
    if 0 in numbers and 1 in numbers:
        return [999] #oh no our sort is wrong
    return sorted(numbers)

# defined strategy
list_strategy = st.lists(
    st.integers(min_value=-5, max_value=5),
    min_size=1,
    max_size=10
)

# ignore
seen_inputs = []

#ignore
def check_sort(numbers: List[int]):
    seen_inputs.append(numbers[:])  
    note(f"Checking example: {numbers}")
    assert buggy_sort(numbers) == sorted(numbers)

# --- Settings ---
@settings(verbosity=2, max_examples=1000, deadline=None)
def run_find():
    print("Searching for example")
    failing_example = find(list_strategy, check_sort)
    print(f" {failing_example}")

run_find()

In [ ]:
import matplotlib.pyplot as plt

sizes = [len(x) for x in seen_inputs]
plt.figure(figsize=(10, 5))
plt.plot(sizes, marker='o', linestyle='-', color='blue', label='Input size')
plt.xlabel('Test number')
plt.ylabel('Input size (length of list)')
plt.title(' Shrinking Behavior in Hypothesis')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

You might note it shrank it down to the simplest possible case that still causes the error.

#### Replaying Failures

When a test fails, Hypothesis saves the failing example. You can use the `@example` decorator to add it as a permanent regression test case, ensuring the bug never comes back.

In [ ]:
from hypothesis import given, example


@given(st.lists(st.integers()))
@example([0, 1])  
def test_buggy_sort(numbers):
    try:
        assert buggy_sort(numbers) == sorted(numbers)
    except AssertionError:
        print(f"Test failed for input: {numbers}")

test_buggy_sort()

print("Test with @example ran.")


### Note
Hypothesis database may be invalidated when using different Hypothesis versions or when changing the source code of test functions. Instead of relying on the Hypothesis to replay the last failure from the database, it may be preferred to use `@example` in these cases.

## Filtering and Transformations (`.filter()`, `assume`, `.map()`) — **Pritam**

Hypothesis provides several ways to narrow down the set of possible examples to generate.

In [ ]:
from hypothesis import assume

count_examples = {"filter": 0, "assume": 0, "early return": 0}
count_pass = {"filter": 0, "assume": 0, "early return": 0}

@settings(max_examples=50)
@given(st.integers().filter(lambda x: x % 2 == 0), st.integers().filter(lambda x: x % 2 == 0))
def test_filter(a, b):
    count_examples["filter"] += 1

    assert (a+b) % 2 == 0

    count_pass["filter"] += 1

@settings(max_examples=50)
@given(st.integers(), st.integers())
def test_assume(a, b):
    count_examples["assume"] += 1

    assume(a % 2 == 0 and b % 2 == 0)
    assert (a+b) % 2 == 0

    count_pass["assume"] += 1

@settings(max_examples=50)
@given(st.integers(), st.integers())
def test_early_return(a, b):
    count_examples["early return"] += 1

    if a % 2 != 0 or b % 2 != 0:
        return
    assert (a+b) % 2 == 0

    count_pass["early return"] += 1

test_filter()
test_assume()
test_early_return()

print(f"{'method':15s} | {'examples created':20s} | {'success rate':>11s}")
print("="*60)
for method in "filter", "assume", "early return":
    print(f"{method:15s} | {count_examples[method]:20d} | {count_pass[method]:10d}/50")

In [ ]:
import time

@settings(max_examples=500)
@given(st.integers().map(lambda x: x*2), st.integers().map(lambda x: x*2))
def test_mapping(a, b):
    assert (a+b) % 2 == 0

@settings(max_examples=500)
@given(st.integers().filter(lambda x: x % 2 == 0), st.integers().filter(lambda x: x % 2 == 0))
def test_filtering(a, b):
    assert (a+b) % 2 == 0

for method, fn in ("mapping", test_mapping), ("filtering", test_filtering):
    start_time = time.perf_counter()
    fn()
    end_time = time.perf_counter()
    elapsed = end_time - start_time
    print(f"{method:8s} | time: {elapsed:8.4f}s")

### Takeaways
1. A successful example is one where an assert does not fail which is why early return generates exactly 50 examples that are not considered failures.
2. `.assume()` will not make progress towards max examples if assumption fails, but will still invoke the function (think about side effects if assumption is in middle of body)
3. Should prefer `.filter()` to `.assume()` and avoid early return when using Hypothesis.
4. Consider using mapping over filtering for potential perfomance gains.

Hypothesis also claims that simple `.filter()` can be sometimes be optimized beyond trivial rejection sampling unlike `.assume()`

## 4. Custom and Composite Strategies

What if you need to generate custom objects? The `@composite` decorator lets you build new strategies from existing ones.

In [ ]:
from hypothesis.strategies import composite
from dataclasses import dataclass

@dataclass
class User:
    id: int
    name: str

@composite
def user_strategy(draw):
    """A strategy to generate User objects."""
    # `draw` works like `data.draw()` from the previous example
    user_id = draw(st.integers(min_value=1))
    user_name = draw(st.text(min_size=3, max_size=20))
    return User(id=user_id, name=user_name)

@given(user=user_strategy())
def test_user_creation(user):
    print(f"Generated User: {user}")
    assert isinstance(user, User)
    assert user.id > 0
    assert len(user.name) >= 3

test_user_creation()

## 5. Stateful Testing

Hypothesis can even test stateful systems by generating sequences of actions and checking that invariants (rules that should always be true) hold.

Let's test a simple `SimpleStack` class.

In [ ]:
from hypothesis.stateful import RuleBasedStateMachine, rule, precondition

class SimpleStack:
    """A basic stack implementation."""
    def __init__(self):
        self._items = []
    
    def push(self, item):
        self._items.append(item)
        
    def pop(self):
        if not self._items:
            raise IndexError("pop from empty list")
        return self._items.pop()
    
    def is_empty(self):
        return not self._items
    
    @property
    def size(self):
        return len(self._items)

class StackStateMachine(RuleBasedStateMachine):
    """Defines the rules for testing our stack."""
    def __init__(self):
        super().__init__()
        self.stack = SimpleStack()
        self.model = [] # A simple list to model the stack's behavior
        
    @rule(item=st.integers())
    def push_item(self, item):
        """Rule for pushing an item."""
        self.stack.push(item)
        self.model.append(item)
        print(f"Pushed {item}")
        
    @rule()
    @precondition(lambda self: not self.stack.is_empty()) # Only run pop if not empty
    def pop_item(self):
        """Rule for popping an item."""
        popped_stack = self.stack.pop()
        popped_model = self.model.pop()
        print(f"Popped {popped_stack}")
        assert popped_stack == popped_model
    
    @rule()
    def check_invariants(self):
        """This rule checks properties that should always be true."""
        print("Checking invariants...")
        assert self.stack.size == len(self.model)
        assert self.stack.is_empty() == (not self.model)

# To run this, Hypothesis would execute a sequence of the rules defined above.
TestStack = StackStateMachine.TestCase
TestStack.runTest = lambda self: None # Suppress unittest's default runner

test_case = TestStack()
test_case.execute_step = test_case.execute_step
print("Running stateful test...")
# Manually run a few steps for demonstration
try:
    for i in range(10): # Run 10 random steps
        test_case.execute_step(test_case.steps.pop(0))
except (IndexError, AttributeError):
    pass # Stop if we run out of generated steps
print("\nStateful test finished.")